In [1]:
import pandas as pd

In [6]:
df3=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/3and4G_patents (1).csv")

In [7]:
df3.head()

,Text,Gen
0,Multi-functional third generation digital poli...,3G
1,Time division-synchronous code division multip...,3G
2,Time division-synchronous code division multip...,3G
3,Wideband code division multiple access and tim...,3G
4,Mobile terminal has double module communicatio...,3G


In [8]:
#keeping only 4G columns
df4=df3[df3["Gen"]!="3G"]

In [9]:
import nltk
from nltk.corpus import stopwords
import spacy
nlp=spacy.load("en_core_web_sm")
import re

pos_tags= ['NOUN', 'VERB', 'ADJ','PROPN']

In [10]:
import re
stop_words = nlp.Defaults.stop_words
def preprocess(text):
  text=text.lower()
  text=re.sub(r'[^A-Za-z]',' ', text)
  doc=nlp(text)
  lemmatized_words=[word.lemma_ for word in doc if word.pos_ in pos_tags]
  cleaned_words=[word for word in lemmatized_words if word not in stop_words]
  return ' '.join(cleaned_words)

In [12]:
import tqdm
tqdm.tqdm.pandas()

In [13]:
df4['clean_text']=df4['Text'].progress_apply(preprocess)

100%|██████████| 43857/43857 [38:32<00:00, 18.96it/s]
<ipython-input-13-e33b30e7517b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['clean_text']=df4['Text'].progress_apply(preprocess)


In [15]:
#convert text to dqm using countvectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer=CountVectorizer(max_df=0.95, min_df=2,stop_words='english')
dtm=vectorizer.fit_transform(df4['clean_text'])

In [16]:
#applying LDA

lda = LatentDirichletAllocation(n_components=20, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=20, random_state=42)

In [17]:
def display_topics(model, feature_names, no_top_words):
    for idx, topic in enumerate(model.components_):
        print(f"Topic {idx + 1}: ", end="")
        print(" | ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
feature_names = vectorizer.get_feature_names_out()
display_topics(lda, feature_names, no_top_words)

Topic 1: information | vehicle | device | mobile | terminal | position | display | server | communication | phone
Topic 2: channel | control | information | downlink | physical | transmission | method | transmit | feedback | block
Topic 3: signal | power | interference | method | transmission | channel | receive | base | receiver | output
Topic 4: method | parameter | lte | value | obtain | involve | drawing | accord | base | determine
Topic 5: datum | packet | data | method | transmit | layer | transmission | protocol | receive | buffer
Topic 6: signal | frequency | method | time | symbol | synchronization | sequence | value | code | description
Topic 7: service | broadcast | video | location | content | provide | method | user | multicast | network
Topic 8: station | base | communication | unit | terminal | wireless | mobile | apparatus | control | information
Topic 9: network | mobile | service | method | device | access | communication | traffic | user | provide
Topic 10: message |